In [1]:
import wrds
import pandas as pd

In [2]:
conn = wrds.Connection()

Enter your WRDS username [senior]: ctyy
Enter your password: ·············


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
Loading library list...
Done


In [3]:
conn.list_libraries().sort()
conn.list_libraries()

['aha',
 'aha_sample',
 'ahasamp',
 'audit',
 'auditsmp',
 'auditsmp_all',
 'bank',
 'blab',
 'block',
 'block_all',
 'boardex',
 'boardex_trial',
 'boardsmp',
 'bvd',
 'bvdsamp',
 'calcbench_trial',
 'calcbnch',
 'cboe',
 'cboe_all',
 'centris',
 'ciq',
 'ciq_capstrct',
 'ciq_common',
 'ciqsamp',
 'ciqsamp_common',
 'ciqsamp_transcripts',
 'cisdm',
 'cisdmsmp',
 'clrvt',
 'clrvtsmp',
 'columnar',
 'comp',
 'comp_bank',
 'comp_bank_daily',
 'comp_execucomp',
 'comp_global',
 'comp_global_daily',
 'comp_na_annual_all',
 'comp_na_daily_all',
 'comp_na_monthly_all',
 'comp_segments_hist',
 'comp_segments_hist_daily',
 'compa',
 'compb',
 'compbd',
 'compdcur',
 'compg',
 'compgd',
 'comph',
 'compm',
 'compmcur',
 'compnad',
 'compsamp',
 'compsamp_all',
 'compsamp_snapshot',
 'compseg',
 'compsegd',
 'compsnap',
 'comscore',
 'contrib',
 'contrib_ceo_turnover',
 'contrib_char_returns',
 'contrib_general',
 'contrib_intangible_value',
 'contrib_kpss',
 'contrib_liva',
 'contrib_shale',
 '

In [7]:
# 通常財務資料都在comp抓 也可以去wharton的網站看
# comp應該是屬於全部comp的都抓 像comp_na_daily_all只有包含daily的 雖然不確定真正差異在哪
conn.list_tables(library='comp_na_daily_all')

['aco_amda',
 'aco_imda',
 'aco_indfnta',
 'aco_indfntq',
 'aco_indfntytd',
 'aco_indsta',
 'aco_indstq',
 'aco_indstytd',
 'aco_notesa',
 'aco_notesq',
 'aco_notessa',
 'aco_notesytd',
 'aco_pnfnda',
 'aco_pnfndq',
 'aco_pnfndytd',
 'aco_pnfnta',
 'aco_pnfntq',
 'aco_pnfntytd',
 'aco_transa',
 'aco_transq',
 'aco_transsa',
 'aco_transytd',
 'adsprate',
 'asec_amda',
 'asec_imda',
 'asec_notesa',
 'asec_notesq',
 'asec_transa',
 'asec_transq',
 'chars',
 'co_aacctchg',
 'co_aaudit',
 'co_adesind',
 'co_adjfact',
 'co_afnd1',
 'co_afnd2',
 'co_afnddc1',
 'co_afnddc2',
 'co_afntind1',
 'co_afntind2',
 'co_ainvval',
 'co_amkt',
 'co_busdescl',
 'co_cotype',
 'co_filedate',
 'co_fortune',
 'co_hgic',
 'co_iacctchg',
 'co_iaudit',
 'co_idesind',
 'co_ifndq',
 'co_ifndsa',
 'co_ifndytd',
 'co_ifntq',
 'co_ifntsa',
 'co_ifntytd',
 'co_imkt',
 'co_industry',
 'co_ipcd',
 'co_mthly',
 'co_offtitl',
 'company',
 'currency',
 'dd_group',
 'dd_group_xref',
 'dd_item',
 'dd_package',
 'ecind_desc',

In [5]:
# Extract first 5 obs from comp.company

company = conn.get_table(library='comp', table='co_afnd1', obs=5)
company

,gvkey,indfmt,datafmt,consol,popsrc,acchg,accli,acco,accrt,acdo,...,lrv,ls,lse,lst,lt,ltdch,ltdlch,ltlo,lul3,datadate
0,001000,INDL,STD,C,D,None,None,None,None,None,...,None,None,NaN,None,NaN,None,None,None,None,1961-12-31
1,001000,INDL,STD,C,D,None,None,None,None,None,...,None,None,NaN,None,NaN,None,None,None,None,1962-12-31
2,001000,INDL,STD,C,D,None,None,None,None,None,...,None,None,NaN,None,0.345,None,None,None,None,1963-12-31
3,001000,INDL,STD,C,D,None,None,None,None,None,...,None,None,1.416,None,0.809,None,None,None,None,1964-12-31
4,001000,INDL,STD,C,D,None,None,None,None,None,...,None,None,2.310,None,1.818,None,None,None,None,1965-12-31


In [8]:
query = """
select a.at, a.ebit, b.ni, b.oancf, b.opeps, b.teq, b.xrd, c.sic, c.tic, d.conml, a.gvkey, a.datadate

from comp.co_afnd1 a
inner join comp.co_afnd2 b
on a.gvkey = b.gvkey
and a.datadate = b.datadate

inner join comp.namesq c
on a.gvkey = c.gvkey

inner join comp.company d
on a.gvkey = d.gvkey

WHERE a.datadate>='01/01/2010'
and a.datadate<='12/31/2020'
and c.sic like '737%%'
and b.xrd is not null
and b.xrd > 0
and a.ebit is not null

"""
df_sql = conn.raw_sql(query)
df_sql

,at,ebit,ni,oancf,opeps,teq,xrd,sic,tic,conml,gvkey,datadate
0,2447.200,134.800,58.000,246.800,0.54,1473.500,457.500,7370,ADSK,AUTODESK INC,001878,2010-01-31
1,97.175,7.979,5.704,-1.876,0.25,72.280,6.722,7372,AMSWA,American Software Inc,001562,2010-04-30
2,104.832,9.412,7.364,3.066,0.29,74.056,7.388,7372,AMSWA,American Software Inc,001562,2011-04-30
3,116.553,16.168,11.343,16.281,0.43,83.030,8.226,7372,AMSWA,American Software Inc,001562,2012-04-30
4,113.070,13.784,10.411,19.128,0.38,83.244,8.882,7372,AMSWA,American Software Inc,001562,2013-04-30
...,...,...,...,...,...,...,...,...,...,...,...,...
5445,43.833,1.911,1.936,3.236,0.03,32.666,0.027,7372,ARBK,Argo Blockchain Plc,327832,2020-12-31
5446,1012.720,-358.088,-348.535,-176.558,-1.26,391.717,105.160,7370,SNOW,Snowflake Inc,339965,2020-01-31
5447,4.813,-8.396,-10.684,-4.483,-0.83,2.074,2.742,7372,IDAI,T Stamp Inc,345764,2020-12-31
5448,108.754,38.322,16.851,33.588,0.79,59.574,3.434,7370,KARO,Karooooo Ltd,347085,2019-02-28


#Get Company Annual Item(AL)

In [11]:
df_sql = conn.raw_sql("""select gvkey, ebit ,act, datadate from comp.co_afnd1 WHERE gvkey = '001690'""")#gvkey 001690 = AAPL
df_sql.head()
# df_sql.to_csv("AL_gvkey001690_Company Annual Item.csv",index=False)

,gvkey,ebit,act,datadate
0,001690,23.586,54.106,1980-09-30
1,001690,NaN,NaN,2021-09-30
2,001690,66.143,227.104,1981-09-30
3,001690,102.209,317.075,1982-09-30
4,001690,129.639,468.993,1983-09-30


#Get Company Annual Item(MZ)(費用、收入)

In [12]:
#xrd = Research and Development Expense
df_sql = conn.raw_sql("""select gvkey, xrd, datadate from comp.co_afnd2 WHERE gvkey = '001690'""")#gvkey 001690 = AAPL
df_sql.head()
# df_sql.to_csv("MZ_gvkey001690_Company Annual Item.csv",index=False)

,gvkey,xrd,datadate
0,001690,NaN,2021-09-30
1,001690,60.040,1983-09-30
2,001690,71.136,1984-09-30
3,001690,NaN,2014-09-30
4,001690,7.282,1980-09-30
